<a href="https://colab.research.google.com/github/motorlearner/neuromatch/blob/main/laquitaine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# imports
# import os
import requests
import pandas as pd
import numpy as np

In [35]:
# @title Read Data
# ------------------------------------------------------------------------------

# fetch data
url = "https://github.com/steevelaquitaine/projInference/raw/gh-pages/data/csv/data01_direction4priors.csv"
try:
  RequestAPI = requests.get(url)
except requests.ConnectionError:
  print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
else:
  if RequestAPI.status_code != requests.codes.ok:
    print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
  else:
    with open("data01_direction4priors.csv", "wb") as fid:
      fid.write(RequestAPI.content)

# read data
df = pd.read_csv("data01_direction4priors.csv")

In [49]:
# @title Helpers: Process Data
# ---------------------------------------------------------------------------- #


# COORDINATE TRANSFORMS ------------------------------------------------------ #

def cart2pol(x:np.array, y:np.array):
  """
  Convert cartesian `(x,y)` to polar `(deg,mag)`.

  Args:
      `x`: scalar or array of x-coordinates
      `y`: scalar or array of y-coordinates

  Returns:
      Tuple of `(deg,mag)`, where `deg` is in the interval
      [0,360) counterclockwise from the positive x axis.
  """
  # checks
  # compute
  deg = (np.degrees(np.arctan2(y,x)) + 360) % 360
  mag = np.hypot(x,y)
  return deg,mag


def cart2deg(x:np.array, y:np.array):
  return cart2pol(x,y)[0]


def cart2mag(x:np.array, y:np.array):
  return cart2pol(x,y)[1]


def pol2cart(deg: np.array, mag: np.array):
  """
  Convert polar `(deg,mag)` to cartesian `(x,y)`.

  Args:
      `deg`: scalar or array of angles in degrees [0,360)
      `mag`: scalar or array of magnitudes

  Returns:
      Tuple of `(x, y)` coordinates.
  """
  # Convert degrees to radians
  rad = np.radians(deg)
  # Compute x and y
  x = mag * np.cos(rad)
  y = mag * np.sin(rad)
  return x, y

# CIRCULAR DISTANCE ---------------------------------------------------------- #

def circdiff(angle:np.array, reference:np.array):
  """
  Compute the signed minimal circular distance from `angle` to `reference`.

  Args:
      `angle`: scalar or array of angles in degrees [0, 360)
      `reference`: scalar reference angle in degrees [0, 360)

  Returns:
      Signed circular distance in degrees (-180, 180].
  """
  return ((angle - reference + 180) % 360) - 180

In [51]:
df['estimate_deg'], df['estimate_mag'] = cart2pol(df['estimate_x'], df['estimate_y'])
df['estimate_degdelta'] = circdiff(df['estimate_deg'], df['prior_mean'])

df.head(20)

,trial_index,trial_time,response_arrow_start_angle,motion_direction,motion_coherence,estimate_x,estimate_y,reaction_time,raw_response_time,prior_std,prior_mean,subject_id,experiment_name,experiment_id,session_id,run_id,estimate_deg,estimate_mag,estimate_degdelta
0,1,0.000000,NaN,225,0.12,-1.749685,-1.785666,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,225.583113,2.5,0.583113
1,2,2.730730,NaN,225,0.12,-1.819693,-1.714269,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,223.291282,2.5,-1.708718
2,3,4.913950,NaN,235,0.06,-1.562674,-1.951422,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,231.312691,2.5,6.312691
3,4,6.997296,NaN,225,0.06,-1.601388,-1.919781,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,230.166776,2.5,5.166776
4,5,9.097130,NaN,215,0.24,-1.639461,-1.887371,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,229.020860,2.5,4.020860
5,6,11.030394,NaN,205,0.24,-1.819693,-1.714269,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,223.291282,2.5,-1.708718
6,7,13.080369,NaN,235,0.06,-1.713624,-1.820300,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,226.729029,2.5,1.729029
7,8,15.146855,NaN,215,0.12,-1.919213,-1.602068,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,219.853535,2.5,-5.146465
8,9,17.163580,NaN,225,0.12,-1.601388,-1.919781,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,230.166776,2.5,5.166776
9,10,19.496723,NaN,225,0.12,-1.401729,-2.070062,NaN,NaN,10,225,1,data01_direction4priors,11,1,1,235.896354,2.5,10.896354
